In [2]:
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths

import numpy as np
import random
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline

c:\Users\abdel\.conda\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\abdel\.conda\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\abdel\.conda\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
def seed_everything(seed=42):
    # Random state is steady among the data
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

def find_indices(array, value):
    return np.where(array == value)[0]

# Read test dataset and the number of elements in each class

In [14]:
data_test = np.load('Dataset5_raw_test.npz')
image_test, image_label_test = data_test['image'].astype(np.uint8), data_test['image_label'].astype(np.int8)
unique, counts = np.unique(image_label_test, return_counts=True)
counts

array([ 838, 1203, 2039, 1480,  980], dtype=int64)

# Read train and valid dataset and make it balanced

In [4]:
data_train = np.load('Dataset5_raw_train.npz')
image_train, image_label_train = data_train['image'].astype(np.uint8), data_train['image_label'].astype(np.int8)

In [5]:
X_train, y_train = image_train[:2326], image_label_train[:2326]
for i in range(1, 5):
    seed_everything()
    train1_ind = find_indices(image_label_train, i)
    np.random.shuffle(train1_ind)
    X_train = np.concatenate([X_train, image_train[train1_ind[:2326]]])
    y_train = np.concatenate([y_train, image_label_train[train1_ind[:2326]]])

unique, counts = np.unique(y_train, return_counts=True)
counts

array([2326, 2326, 2326, 2326, 2326], dtype=int64)

In [6]:
data_val = np.load('Dataset5_raw_val.npz')
image_val, image_label_val = data_val['image'].astype(np.uint8), data_val['image_label'].astype(np.int8)
unique, counts = np.unique(image_label_val, return_counts=True)
counts

array([ 836, 1202, 2038, 1479,  979], dtype=int64)

In [7]:
X_val, y_val = image_val[:836], image_label_val[:836]
for i in range(1, 5):
    seed_everything()
    val1_ind = find_indices(image_label_val, i)
    np.random.shuffle(val1_ind)
    X_val = np.concatenate([X_val, image_val[val1_ind[:836]]])
    y_val = np.concatenate([y_val, image_label_val[val1_ind[:836]]])
    
unique, counts = np.unique(y_val, return_counts=True)
counts

array([836, 836, 836, 836, 836], dtype=int64)

In [8]:
#Combining
image_total = np.concatenate([X_train, X_val])
label_total = np.concatenate([y_train, y_val])
unique, counts = np.unique(label_total, return_counts=True)
print(counts)
# del X_train, X_val, y_train, y_val, image_train, image_label_train, image_val, image_label_val

[3162 3162 3162 3162 3162]


In [ ]:
for i in range(0, 5):
    print('Class', i)
    seed_everything()
    val1_ind = find_indices(label_total, i)
    np.random.shuffle(val1_ind)
    if i == 0:
        X_total = image_total[val1_ind[:1962]]
        y_total = label_total[val1_ind[:1962]]
    else:
        X_total = np.concatenate([X_total, image_total[val1_ind[:1962]]])
        y_total = np.concatenate([y_total, label_total[val1_ind[:1962]]])
    
unique, counts = np.unique(y_total, return_counts=True)
counts

# Soft Augmentation

In [15]:
import albumentations as A
COUNT_TEST = 1

#### Test dataset resizing only.
for img, y in zip(image_test, image_label_test):
  # img = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
  y_str = str(y[0])
  resizeTransform = A.Compose([A.Resize(height=224, width=224, interpolation=1)], p=1)
  new_img = resizeTransform(image = img)['image']
  folder_to_save_img = "Images/Soft Aug/Soft_Aug 25-75/Test"
  name_of_augmented_file = f"{COUNT_TEST}.jpg"
  cv2.imwrite(f"{folder_to_save_img}/{y_str}/{name_of_augmented_file}", new_img)
  COUNT_TEST+=1

In [16]:
COUNT=0
for img, y in zip(X_total, y_total): #Imbalanced: image_val, image_label_val, Balanced: X_val, y_val
  y_str = str(y[0])
  for i in range(2):
    resizeTransform = A.Compose([A.Resize(height=224, width=224, interpolation=1)], p=1)
    new_img = resizeTransform(image = img)['image']
    transform = A.Compose(
    [
      A.ImageCompression(quality_lower = 92, quality_upper=100, p=0.7),
      A.HorizontalFlip(p = 0.5),
      A.OneOf([
        A.Rotate(limit=180, interpolation=1, border_mode=4, always_apply=False, p=0.7),
        A.Rotate(limit=180, interpolation=1, border_mode=0, always_apply=False, p=0.3),
      ], p = 0.7),
      A.ShiftScaleRotate(p = 0.5),
    ],
    p = 0.9)
    aug = transform(image = new_img)["image"] 
    folder_to_save_img = "Images/Soft Aug/Soft_Aug 25-75/Train"
    name_of_augmented_file = f"{COUNT}.jpg"
    if i == 0:
      image_to_save = new_img
    else:
      image_to_save = aug
    cv2.imwrite(f"{folder_to_save_img}/{y_str}/{name_of_augmented_file}", image_to_save)
    COUNT+=1
print("Done Validation")

Done Validation


# Number of elements in each class after the augmentation 

In [17]:
sum = 0
for i in range(5):
    sum = sum + len(os.listdir(f'Images\Soft Aug\Soft_Aug 25-75/Train/{i}'))
    print(f'{i}: ')
    print(len(os.listdir(f'Images\Soft Aug\Soft_Aug 25-75/Train/{i}')))
print(f'total: {sum}')


0: 
3924
1: 
3924
2: 
3924
3: 
3924
4: 
3924
total: 19620
